In [ ]:
!pip install opencv-python
!pip install tensorflow

In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from collections import defaultdict

# Configure GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU detected and configured.")
else:
    print("No GPU detected.")

dataset_dir = '/content/drive/MyDrive/Colab Notebooks/deep learning/breast cancer/cancer dataset'
img_size = (224, 224)
limit_per_class = 5000

# Load img
data = []
labels = []
class_counts = defaultdict(int)

for subdir in sorted(os.listdir(dataset_dir)):
    subdir_path = os.path.join(dataset_dir, subdir)
    if os.path.isdir(subdir_path):
        for img_name in sorted(os.listdir(subdir_path)):
            if class_counts[subdir] >= limit_per_class:
                break
            img_path = os.path.join(subdir_path, img_name)
            try:
                img = load_img(img_path, target_size=img_size)
                img_array = img_to_array(img) / 255.0
                data.append(img_array)
                labels.append(subdir)
                class_counts[subdir] += 1
            except Exception as e:
                print(f"Skipping {img_path}: {e}")

print(f"Images loaded per class: {dict(class_counts)}")

data = np.array(data, dtype='float32')
labels = np.array(labels)

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
num_classes = len(le.classes_)
labels = to_categorical(labels_encoded, num_classes=num_classes)

#split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

# Load VGG16 and freeze
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# model
regularizer = tf.keras.regularizers.l2(0.001)

model = Sequential([
    base_model,
    Flatten(name="flatten", trainable=True),
    Dense(1024, activation='relu', kernel_regularizer=regularizer),
    Dropout(0.7),
    Dense(num_classes, activation='softmax', kernel_regularizer=regularizer)
])

# Compile
model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# earlystopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train
print("Training model...")
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate
score = model.evaluate(X_test, y_test, verbose=0)
print(f'\nTest Accuracy: {score[1]*100:.2f}%')
print(f'Final Training Accuracy: {history.history["accuracy"][-1]*100:.2f}%')
print(f'Final Validation Accuracy: {history.history["val_accuracy"][-1]*100:.2f}%')


✅ GPU detected and configured.
Images loaded per class: {'0_NoCancer': 1180, '1_Cancer': 1178}
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
🔁 Training model...
Epoch 1/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.5652 - loss: 3.8816 - val_accuracy: 0.7373 - val_loss: 1.4491
Epoch 2/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 37s 419ms/step - accuracy: 0.7428 - loss: 1.3397 - val_accuracy: 0.7811 - val_loss: 1.0089
Epoch 3/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 428ms/step - accuracy: 0.7473 - loss: 1.0198 - val_accuracy: 0.7655 - val_loss: 0.9143
Epoch 4/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 440ms/step - accuracy: 0.7678 - loss: 0.8917 - val_accuracy: 0.7740 - val_loss: 0.8792
Epoch 5/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 20s 416ms/step - accuracy: 0.8157 - loss: 0.7822 - val_accuracy: 0.7839 - val_loss: 0.8058
Epoch 6/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 20s 417ms/step - accuracy: 0.8122 - loss: 0.7371 - val_accuracy: 0.7938 - val_loss: 0.7753
Epoch 7/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 22s 498ms/step - accuracy